In [1]:
import attr
import numpy as np
from thermo import Chemical
import warnings

from gibbs.models.ceos import PengRobinson, SoaveRedlichKwong
from gibbs.mixture import Mixture
from gibbs.stability_analysis import stability_test
from gibbs.equilibrium import calculate_equilibrium
from gibbs.utilities import convert_bar_to_Pa, convert_atm_to_Pa

warnings.filterwarnings('ignore')

In [2]:
@attr.s(auto_attribs=True)
class NichitaPR:
    mixture: Mixture
    bip: np.ndarray

    @property
    def model(self):
        return PengRobinson(
            mixture=self.mixture,
            bip=self.bip
        )

    @property
    def number_of_components(self):
        return len(self.mixture.z)

    def fugacity(self, P, T, z):
        Z_factor = self.calculate_Z(P, T, z)
        return self.model.calculate_fugacity(P, T, z, Z_factor)

    def calculate_Z(self, P, T, z):
        Z_factor = self.model.calculate_Z_minimal_energy(P, T, z)
        return Z_factor

## Two-phase vapor-liquid equilibria

### Eight-component mixture

In [3]:
methane = Chemical('methane')
ethane = Chemical('ethane')
propane = Chemical('propane')
nbutane = Chemical('n-butane')
npentane = Chemical('n-pentane')
nhexane = Chemical('n-hexane')

In [4]:
z = np.array([0.6883, 0.0914, 0.0460, 0.0333, 0.0139, 0.0152, 0.0896, 0.0222])
omegas = np.array([methane.omega, ethane.omega, propane.omega, nbutane.omega, npentane.omega, nhexane.omega, 0.4019, 0.7987])
Tcs = np.array([methane.Tc, ethane.Tc, propane.Tc, nbutane.Tc, npentane.Tc, nhexane.Tc, 606.28, 825.67])
Pcs = np.array([methane.Pc, ethane.Pc, propane.Pc, nbutane.Pc, npentane.Pc, nhexane.Pc, convert_atm_to_Pa(25.42), convert_atm_to_Pa(14.39)])
mixture = Mixture(z, Tcs, Pcs, omegas)
kijs = np.zeros((8, 8))
kijs[0, 6] = kijs[6, 0] = 0.050
kijs[1, 6] = kijs[6, 1] = 0.040
kijs[2, 6] = kijs[6, 2] = 0.010
kijs[0, 7] = kijs[7, 0] = 0.090
kijs[1, 7] = kijs[7, 1] = 0.055
kijs[2, 7] = kijs[7, 2] = 0.010

model = NichitaPR(
    mixture=mixture,
    bip=kijs
)

In [5]:
# NBVAL_IGNORE_OUTPUT
T = 353
P = convert_bar_to_Pa(385)

result = calculate_equilibrium(model, P, T, z, molar_base=1, number_of_trial_phases=2, workers=-1, compare_trial_phases=False)

result.X, result.F, result.reduced_gibbs_free_energy

(array([[0.63883771, 0.09026987, 0.04805661, 0.03626643, 0.01566856,
         0.01757364, 0.11596485, 0.03736234],
        [0.72895376, 0.09234264, 0.04432322, 0.03087542, 0.0124534 ,
         0.01325785, 0.06801342, 0.00978029]]),
 array([0.45031783, 0.54958217]),
 array(15.21199279))

## Three-phase vapor-liquid-liquid equilibria

### Case 4.3.1: Ternary mixture C$_1$/nC$_{16}$/CO$_2$

In [6]:
nhexadecane = Chemical('n-hexadecane')
carbon_dioxide = Chemical('carbon-dioxide')

In [7]:
z = np.array([0.05, 0.05, 0.90])
omegas = np.array([methane.omega, nhexadecane.omega, carbon_dioxide.omega])
Tcs = np.array([methane.Tc, nhexadecane.Tc, carbon_dioxide.Tc])
Pcs = np.array([methane.Pc, nhexadecane.Pc, carbon_dioxide.Pc])
mixture = Mixture(z, Tcs, Pcs, omegas)
kijs = np.array([
    [0.000, 0.078, 0.100],
    [0.078, 0.000, 0.125],
    [0.100, 0.125, 0.000]
])

model = NichitaPR(
    mixture=mixture,
    bip=kijs
)

In [8]:
# NBVAL_IGNORE_OUTPUT
T = 294.3
P = convert_bar_to_Pa(67)

result = calculate_equilibrium(model, P, T, z, molar_base=1, number_of_trial_phases=3, workers=-1)

result.X, result.F, result.reduced_gibbs_free_energy

(array([[4.93204185e-02, 2.97175437e-03, 9.47707827e-01],
        [8.20517447e-02, 9.80671124e-05, 9.17850188e-01],
        [3.07412624e-02, 3.63209638e-01, 6.06049100e-01]]),
 array([0.77339581, 0.09529624, 0.13130795]),
 14.24645804708375)

In [9]:
number_of_intervals = 9
P_range = np.linspace(convert_bar_to_Pa(62), convert_bar_to_Pa(72), number_of_intervals)

F_results = np.array([])
for P in P_range:
    print(f"-> Pressure = {P / 1e6} MPa")
    result = calculate_equilibrium(model, P, T, z, molar_base=1, number_of_trial_phases=3, compare_trial_phases=True, workers=-1)
    F_results = np.append(F_results, result.F)
    print(f"{result.F}")

-> Pressure = 6.2 MPa
[0.13132727 0.86867273]
-> Pressure = 6.325 MPa
[0.86596144 0.13403856]
-> Pressure = 6.45 MPa
[0.86342632 0.13657368]
-> Pressure = 6.575 MPa
[0.13387615 0.54549767 0.32062618]
-> Pressure = 6.7 MPa
[0.13269201 0.51777614 0.34953185]
-> Pressure = 6.825 MPa
[0.67872102 0.18942401 0.13185496]
-> Pressure = 6.95 MPa
[0.86894379 0.13105621]
-> Pressure = 7.075 MPa
[0.13109469 0.86890531]
-> Pressure = 7.2 MPa
[0.13116526 0.86883474]
